In [1]:
from rl_agent import AgentGNN
import gym_zx
import gymnasium as gym
import torch
from torch_geometric.data import Batch, Data
import pyzx as zx

In [2]:
SrH_env = gym.make("zx-v0", qubits=12, depth=None, env_id=1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rl_agent = AgentGNN(None,device=device).to(device)
rl_agent.load_state_dict(
        torch.load("/home/jnogue/qilimanjaro/Copt-cquere/rl-zx/state_dict_2150400model5x70_twoqubits_new.pt", map_location=torch.device("cpu"))
    )  
rl_agent.eval()


AgentGNN(
  (global_attention_critic): AttentionalAggregation(gate_nn=Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  ), nn=Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
  ))
  (critic_gnn): Sequential(
    (0): GATv2Conv(12, 32, heads=1)
    (1): ReLU()
    (2): GATv2Conv(32, 32, heads=1)
    (3): ReLU()
    (4): GATv2Conv(32, 32, heads=1)
    (5): ReLU()
    (6): GATv2Conv(32, 32, heads=1)
    (7): ReLU()
    (8): GATv2Conv(32, 32, heads=1)
    (9): ReLU()
  )
  (actor_gnn): Sequential(
    (0): GATv2Conv(17, 32, heads=1)
    (1): ReLU()
    (2): GATv2Conv(32, 32, heads=1)
    (3): ReLU()
    (4): GATv2Conv(32, 32, heads=1)
    (5): ReLU()
    (6): GATv2Conv(32, 32, heads=1)
    (7): ReLU()
   

In [80]:
obs0, reset_info = SrH_env.reset()
policy_items, value_items = reset_info["graph_obs"]
value_graph = [Data(x=value_items[0], edge_index=value_items[1], edge_attr=value_items[2])]
policy_graph = [Data(x=policy_items[0], edge_index=policy_items[1], edge_attr=policy_items[2], y=policy_items[3])]
next_obs_graph = (Batch.from_data_list(policy_graph), Batch.from_data_list(value_graph))

In [81]:

done = False
while not done:
    
    action, _, _, value, logits, action_ids = rl_agent.get_action_and_value(next_obs_graph, device=device)
    next_obs, reward, done, deprecated, info = SrH_env.step(action_ids.cpu().numpy())
    policy_items, value_items = info["graph_obs"]
    value_graph = [Data(x=value_items[0], edge_index=value_items[1], edge_attr=value_items[2])]
    policy_graph = [Data(x=policy_items[0], edge_index=policy_items[1], edge_attr=policy_items[2], y=policy_items[3])]
    next_obs_graph = (Batch.from_data_list(policy_graph), Batch.from_data_list(value_graph))

Win vs Pyzx:  -1  Episode Gates:  72 Cflow_gates:  33 Episode Len 125 Opt Episode Len 88


In [15]:
SrH_env.pyzx_data

/home/jnogue/miniconda3/envs/cquere/lib/python3.10/site-packages/gymnasium/core.py:297: UserWarning: WARN: env.pyzx_data to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.pyzx_data` for environment variables or `env.get_attr('pyzx_data')` that will search the reminding wrappers.
  logger.warn(


{'qubits': 12,
 'gates': 60,
 'tcount': 8,
 'clifford': 19,
 'CNOT': 26,
 'CZ': 7,
 'had': 5,
 'twoqubits': 33}

In [19]:
data = SrH_env.pyzx_data
print(data)
circuit = SrH_env.final_circuit



{'qubits': 12, 'gates': 60, 'tcount': 8, 'clifford': 19, 'CNOT': 26, 'CZ': 7, 'had': 5, 'twoqubits': 33}


/home/jnogue/miniconda3/envs/cquere/lib/python3.10/site-packages/gymnasium/core.py:297: UserWarning: WARN: env.pyzx_data to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.pyzx_data` for environment variables or `env.get_attr('pyzx_data')` that will search the reminding wrappers.
  logger.warn(
/home/jnogue/miniconda3/envs/cquere/lib/python3.10/site-packages/gymnasium/core.py:297: UserWarning: WARN: env.final_circuit to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.final_circuit` for environment variables or `env.get_attr('final_circuit')` that will search the reminding wrappers.
  logger.warn(


In [65]:
import pyzx as zx
graph = SrH_env.final_circuit.to_graph()
print("Stats after RL-ZX\n", SrH_env.final_circuit.stats())
final_circuit = flow_opt(SrH_env.final_circuit)
print("Stats adding Calum as post processing\n", final_circuit.stats())


Stats after RL-ZX
 Circuit  on 12 qubits with 109 gates.
        8 is the T-count
        101 Cliffords among which
        58 2-qubit gates (50 CNOT, 8 other) and
        6 Hadamard gates.
Stats adding Calum as post processing
 Circuit  on 12 qubits with 102 gates.
        8 is the T-count
        94 Cliffords among which
        53 2-qubit gates (39 CNOT, 14 other) and
        8 Hadamard gates.


In [18]:
circuit.stats()

'Circuit  on 12 qubits with 64 gates.\n        8 is the T-count\n        56 Cliffords among which\n        35 2-qubit gates (23 CNOT, 12 other) and\n        2 Hadamard gates.'

In [75]:
circuit = zx.Circuit.from_qasm_file("/home/jnogue/qilimanjaro/Copt-cquere/rl-zx/cquere/circuits/before/SrH_2.1461_12q.qasm").to_basic_gates()
circ = flow_opt(circuit)
final_circ = full_reduce(circuit)
print(circ.stats())
print(final_circ.stats())


Circuit  on 12 qubits with 60 gates.
        8 is the T-count
        52 Cliffords among which
        33 2-qubit gates (26 CNOT, 7 other) and
        5 Hadamard gates.
Circuit  on 12 qubits with 57 gates.
        8 is the T-count
        49 Cliffords among which
        36 2-qubit gates (31 CNOT, 5 other) and
        4 Hadamard gates.


In [74]:
def basic_optimise(c):
    c1 = zx.basic_optimization(c.copy().split_phase_gates()).to_basic_gates()
    c2 = zx.basic_optimization(c.copy().split_phase_gates(), do_swaps=True).to_basic_gates()
    if c2.twoqubitcount() < c1.twoqubitcount(): return c2 # As this optimisation algorithm is targetted at reducting H-gates, we use the circuit with the smaller 2-qubit gate count here, either using SWAP rules or not.
    return c1
def flow_opt(c):

    c =zx.optimize.basic_optimization(zx.Circuit.from_graph(c.to_graph().copy()).split_phase_gates())
    g = c.to_graph()        
    zx.teleport_reduce(g)
    zx.to_graph_like(g)
    zx.flow_2Q_simp(g)
    c2 = zx.extract_simple(g).to_basic_gates()
    return basic_optimise(c2)

def flow_opt_graph(c):
    c = zx.optimize.basic_optimization(zx.Circuit.from_graph(c.to_graph().copy()).split_phase_gates())
    g = c.to_graph()
    basic_circ = zx.optimize.basic_optimization(zx.Circuit.from_graph(g.copy()).split_phase_gates())
    graph = basic_circ.to_graph()
    zx.simplify.to_gh(graph)
    zx.simplify.spider_simp(graph, quiet=True)
    zx.to_graph_like(graph)
    zx.flow_2Q_simp(graph)
    circuit = zx.extract_simple(graph).to_basic_gates()
    return basic_optimise(circuit)

def full_reduce(c):
    c = zx.optimize.basic_optimization(zx.Circuit.from_graph(c.to_graph().copy()).split_phase_gates())
    g = c.to_graph()        
    zx.teleport_reduce(g)
    zx.to_graph_like(g)
    zx.full_reduce(g)
    circ = zx.extract_circuit(g).to_basic_gates()
    return basic_optimise(circ)
